# Getting started

First, we enable the cluster to scale up. Note that if you run an auto-scaling cluster,
Google will suspend your nodes. Make sure to have the experiment prepared before running the commands.

The following is assumed ready:
* GKE/Kubernetes cluster (see also `terraform/terraform_notebook.ipynb`)
    * 2 nodes pools (default for system & dependencies, experiment pool)
* Docker image (including dataset, to speed-up starting experiments).
    * Within a bash shell
        * Make sure to have the `requirements-cpu.txt` installed (or `requirements-gpu.txt (in a virtual venv/conda environment). You can run `pip3 install -r requirements-cpu.txt`
    * First run the extractor (locally) `python3 -m fltk extractor configs/example_cloud_experiment.json`
        *  This downloads datasets to be included in the docker image.
    * Build the container `DOCKER_BUILDKIT=1 docker build --platform linux/amd64 . --tag gcr.io/$PROJECT_ID/fltk`
    * Push to your gcr.io repository `docker push gcr.io/$PROJECT_ID/fltk`


With that setup, first set some variables used throughout the experiment.


In [1]:
##################
### CHANGE ME! ###
##################
PROJECT_ID="test-bed-fltk-group16-mb"
CLUSTER_NAME="fltk-testbed-cluster"
DEFAULT_POOL="default-node-pool"
EXPERIMENT_POOL="medium-fltk-pool-1"
REGION="us-central1-c"

# In case we do not yet have the credentials/kubeconfig
gcloud container clusters get-credentials $CLUSTER_NAME --region $REGION --project $PROJECT_ID

Fetching cluster endpoint and auth data.
kubeconfig entry generated for fltk-testbed-cluster.


Scale the default-node-pool up.

In [2]:
# These commands might take a while to complete.
gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
     --num-nodes 1 --region $REGION --quiet

Resizing fltk-testbed-cluster...done.


Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-group16-mb/zones/us-central1-c/clusters/fltk-testbed-cluster].


## Preparation
In case you have already tested something or ran another experiment, we have to remove the deployment of the Orchestrator. This will not delete any experiment data, as this persists on one of the ReadWriteMany PVCs.


Currently, the Orchestrator is deployed using a `Deployment` definition, a future version will replace this with a `Deployment` definition, to make this step unnecessary. For experiments this means the following:

1. A single deployment can exist at a single time in a single namespace. This includes 'completed' experiments.
2. For running batches of experiments, a BatchOrchestrator is provided.


ℹ️ This will not remove any data, but if your orchestrator is still/already running experiments, this will stop the deployment. Running training jobs will not be stopped, for this you can use `kubectl`. ConfigMaps created by the Orchestrator (to provide experiment configurations), will not be removed. See the commented code in the cell below.

In [3]:
# If you want to delete all pytorch trainjobs, uncomment the command below.
# kubectl delete pytorchjobs.kubeflow.org --all --namespace test

# If you want to delete all existing configuration map objects in a namespace, run teh command below
# kubectl delete configmaps --all --namespace test

helm uninstall -n test flearner

W1012 15:21:18.401855   12913 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Error: uninstall: Release not loaded: flearner: release: not found


: 1

## Install extractor

Deploy the TensorBoard service and persistent volumes, required for deployment of the orchestrator's chart.

In [4]:
helm upgrade --install -n test extractor ../charts/extractor -f ../charts/fltk-values.yaml \
    --set provider.projectName=$PROJECT_ID

W1012 15:21:26.050659   12939 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Release "extractor" does not exist. Installing it now.
NAME: extractor
LAST DEPLOYED: Wed Oct 12 15:21:27 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Get the FLTK extractors Tensorboard URL by running:

export POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")
echo http://localhost:6006/
kubectl -n test port-forward $POD_NAME 6006:6006


## Define experiment configuration files

Deployment of experiments is currently done through a Helm Deployment. A future release (™️) will rework this to a Job definition, as this allows to re-use the template more easily.


> The `EXPERIMENT_FILE` will contain the description of the experiments
> The `CLUSTER_CONFIG` will contain shared configurations for logging, Orchestrator configuration and replication information.

In [5]:
# EXPERIMENT_FILE="../configs/federated_tasks/example_arrival_config.json"
# EXPERIMENT_FILE="../configs/distributed_tasks/example_arrival_config.json"
# CLUSTER_CONFIG="../configs/example_cloud_experiment.json"

EXPERIMENT_FILE="../project/configs/project_arrival_config_comb_6.json"
CLUSTER_CONFIG="../project/configs/project_cloud_experiment.json"

## Setup experiment variables
Next, we will deploy the experiments.


We provide a configuration file, `charts/fltk-values.yaml`, in here change the values under the `provider` block. Change `projectName` to your Google Cloud Project ID.

```yaml
provider:
    domain: gcr.io
    projectName: CHANGE_ME!
    imageName: fltk:latest
```

We use the `--set-file` flag for `helm`, as currently, Helm does not support using files outside of the chart root directory (in this case `charts/orchestrator`). Using `--set-file` we can dynamically provide these files. See also issue [here](https://github.com/helm/helm/issues/3276)


In [6]:
helm uninstall -n test experiment-orchestrator
helm install -n test experiment-orchestrator ../charts/orchestrator -f ../charts/fltk-values.yaml \
    --set-file orchestrator.experiment=$EXPERIMENT_FILE,orchestrator.configuration=$CLUSTER_CONFIG \
    --set provider.projectName=$PROJECT_ID

W1012 15:22:48.483739   13121 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Error: uninstall: Release not loaded: experiment-orchestrator: release: not found
W1012 15:22:49.142028   13136 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
NAME: experiment-orchestrator
LAST DEPLOYED: Wed Oct 12 15:22:50 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
You successfully launched an experiment configuration on your cluster in test.

N.B. Make sure to collect all data after completing your experiment!
N.B. Re-installing the orchestrator WILL RESULT IN DELETION OF ALL TRAINJOBS and PODS!


In [59]:
# To get logs from the orchestrator
kubectl logs -n test fl-server

W1012 16:08:26.192838   18109 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-12-2022 13:23:01 root         INFO     Loading file config/configuration.fltk.json
No argument path is provided.
10-12-2022 13:23:01 root         INFO     Starting in cluster mode.
10-12-2022 13:23:01 root         INFO     Starting with experiment replication: 0 with seed: 42
10-12-2022 13:23:01 root         INFO     Starting as Orchestrator
10-12-2022 13:23:01 root         INFO     Starting Orchestrator, initializing resources....
10-12-2022 13:23:01 root         INFO     Loading in cluster configuration file
10-12-2022 13:23:01 root         INFO     Pointing configuration to in cluster configuration.
10-12-2022 13:23:01 root         INFO     Starting cluster manager
10-12-2022 13:23:01 ClusterManager INFO     Spinning up cluster manag

10-12-2022 13:26:04 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 13:26:07 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:26:12 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:26:14 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 13:26:17 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:26:22 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:26:24 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 13:26:27 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:26:32 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 

10-12-2022 13:30:03 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:30:08 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 13:30:08 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:30:08 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 13:30:13 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:30:18 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:30:18 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 13:30:23 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:30:28 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 

10-12-2022 13:33:58 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:34:01 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 13:34:03 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:34:08 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:34:11 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 13:34:11 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 13:34:13 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:34:18 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:34:21 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 13:34:23 root      

10-12-2022 13:37:54 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:37:55 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 13:38:00 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:38:05 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:38:05 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 13:38:10 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:38:15 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:38:15 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 13:38:15 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 13:38:20 root      

...

10-12-2022 13:49:42 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:49:46 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 13:49:47 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:49:52 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:49:56 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 13:49:57 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:50:02 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:50:06 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 13:50:08 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 

10-12-2022 13:53:38 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:53:40 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 13:53:43 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:53:48 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:53:50 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 13:53:53 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:53:58 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:54:00 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 13:54:03 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 

10-12-2022 13:57:33 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 13:57:35 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:57:40 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:57:43 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 13:57:45 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:57:50 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:57:53 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 13:57:55 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 13:58:00 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 

10-12-2022 14:01:30 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 14:01:35 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 14:01:37 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 14:01:37 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 14:01:40 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 14:01:45 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 14:01:47 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 14:01:50 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 14:01:56 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 

10-12-2022 14:05:26 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 14:05:31 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 14:05:31 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 14:05:36 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 14:05:41 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 14:05:41 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 14:05:41 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 14:05:46 root         INFO     Waiting for fcdb9c0d-fc40-4463-849b-917da77f859a to complete, 0 pending, 0 arrivals
10-12-2022 14:05:51 ResourceWatchDog INFO     Fetching pod information of cluster...
10-12-2022 14:05:51 root      

In [61]:
# To get logs from learners (example)
kubectl logs -n test trainjob-fcdb9c0d-fc40-4463-849b-917da77f859a-master-0

# To get logs from learners (federated learning)
kubectl logs -n test trainjob-fcdb9c0d-fc40-4463-849b-917da77f859a-worker-0

W1012 16:10:46.870462   18328 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-12-2022 13:25:03 root         INFO     Loading file config/configuration.fltk.json
10-12-2022 13:25:03 root         INFO     Starting in client mode
10-12-2022 13:25:03 root         INFO     Starting with host=trainjob-fcdb9c0d-fc40-4463-849b-917da77f859a-master-0 and port=23456
10-12-2022 13:25:03 root         INFO     Initializing backend for training process: gloo
10-12-2022 13:25:12 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 0
10-12-2022 13:25:12 torch.distributed.distributed_c10d INFO     Rank 0: Completed store-based barrier for key:store_based_barrier_key:1 with 2 nodes.
10-12-2022 13:25:12 root         INFO     Starting Creating client with 0
10-12-2022 13:25:12 Client-0-f

10-12-2022 13:54:18 Client-0-fcdb9c0d-fc40-4463-849b-917da77f859a INFO     [15,     0] loss: 0.002
10-12-2022 13:54:42 Client-0-fcdb9c0d-fc40-4463-849b-917da77f859a INFO     [15,    50] loss: 0.110
10-12-2022 13:55:07 Client-0-fcdb9c0d-fc40-4463-849b-917da77f859a INFO     [15,   100] loss: 0.126
10-12-2022 13:55:31 Client-0-fcdb9c0d-fc40-4463-849b-917da77f859a INFO     [15,   150] loss: 0.123
10-12-2022 13:55:55 Client-0-fcdb9c0d-fc40-4463-849b-917da77f859a INFO     [15,   200] loss: 0.103
10-12-2022 13:56:21 Client-0-fcdb9c0d-fc40-4463-849b-917da77f859a INFO     [16,     0] loss: 0.002
10-12-2022 13:56:46 Client-0-fcdb9c0d-fc40-4463-849b-917da77f859a INFO     [16,    50] loss: 0.104
10-12-2022 13:57:09 Client-0-fcdb9c0d-fc40-4463-849b-917da77f859a INFO     [16,   100] loss: 0.120
10-12-2022 13:57:34 Client-0-fcdb9c0d-fc40-4463-849b-917da77f859a INFO     [16,   150] loss: 0.113
10-12-2022 13:57:58 Client-0-fcdb9c0d-fc40-4463-849b-917da77f859a INFO     [16,   200] loss: 0.099
10-12-2022

10-12-2022 13:37:01 Client-1-fcdb9c0d-fc40-4463-849b-917da77f859a INFO     [6,   150] loss: 0.217
10-12-2022 13:37:25 Client-1-fcdb9c0d-fc40-4463-849b-917da77f859a INFO     [6,   200] loss: 0.209
10-12-2022 13:37:52 Client-1-fcdb9c0d-fc40-4463-849b-917da77f859a INFO     [7,     0] loss: 0.004
10-12-2022 13:38:16 Client-1-fcdb9c0d-fc40-4463-849b-917da77f859a INFO     [7,    50] loss: 0.208
10-12-2022 13:38:40 Client-1-fcdb9c0d-fc40-4463-849b-917da77f859a INFO     [7,   100] loss: 0.209
10-12-2022 13:39:04 Client-1-fcdb9c0d-fc40-4463-849b-917da77f859a INFO     [7,   150] loss: 0.202
10-12-2022 13:39:29 Client-1-fcdb9c0d-fc40-4463-849b-917da77f859a INFO     [7,   200] loss: 0.193
10-12-2022 13:39:54 Client-1-fcdb9c0d-fc40-4463-849b-917da77f859a INFO     [8,     0] loss: 0.004
10-12-2022 13:40:19 Client-1-fcdb9c0d-fc40-4463-849b-917da77f859a INFO     [8,    50] loss: 0.192
10-12-2022 13:40:43 Client-1-fcdb9c0d-fc40-4463-849b-917da77f859a INFO     [8,   100] loss: 0.196
10-12-2022 13:41:07 

## Copy experiment results from the extractor

Extractor holds the experiment results in the format that can be processedby TensorBoard.
In order to download it to the local machine, execute:

In [58]:
EXTRACTOR_POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")

kubectl cp -n test $EXTRACTOR_POD_NAME:/opt/federation-lab/logging ../logging

W1012 16:07:48.283887   18044 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
W1012 16:07:48.900867   18058 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
tar: Removing leading `/' from member names


# Cleanup

## Removing orchestrator

In [62]:
helm uninstall -n test experiment-orchestrator

W1012 16:11:28.981874   18389 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled


## Removing extractor

IMPORTANT: Removing extractor chart will result in deleting the already collected experiment results, stored in the NFS!

In [63]:
helm uninstall extractor -n test

W1012 16:11:41.749574   18458 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "extractor" uninstalled


## Wrapping up

To scale down the cluster nodepools, run the cell below. This will scale the node pools down and remove all the experiments deployed (on the cluster).

1. Experiments cannot be restarted.
2. Experiment logs will not persist deletion.


In [65]:
# This will remove all information and logs as well.
kubectl delete pytorchjobs.kubeflow.org --all-namespaces --all

gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
    --num-nodes 0 --region $REGION --quiet

gcloud container clusters resize $CLUSTER_NAME --node-pool $EXPERIMENT_POOL \
    --num-nodes 0 --region $REGION --quiet

W1012 16:15:33.728567   18680 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
No resources found
Resizing fltk-testbed-cluster...done.                                          
Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-group16-mb/zones/us-central1-c/clusters/fltk-testbed-cluster].
Resizing fltk-testbed-cluster...done.                                          
Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-group16-mb/zones/us-central1-c/clusters/fltk-testbed-cluster].
